## Zadanie domowe: Projektowanie filtru metodą okien

Zauważone w zadaniu z filtracją artefakty są efektem zastosowania filtru idealnego (wycięcia odpowiednich częstotliwości).
Aby je usunąć, należy filtr idealny w dziedzinie przestrzennej przemnożyć przez dwuwymiarowe okno (stworzone na podstawie jednowymiarowego).

1. Zdefiniuj rozmiar filtra (np. 21).

2. Stwórz okno np. Hamminga (`np.Hmming`) o zadanym rozmiarze.

3. Na podstawie tego okna stwórz okno dwuwymiarowe: $F = f^Tf$.
Można to zrealizować prostą instrukcją:
        hanning2d = np.outer(hanning, hanning)

4. Stwórz wzorcowy filtr idealny tak samo jak w poprzednim ćwiczeniu.
Jego rozmiar musi być taki sam jak zdefiniowany w punkcie pierwszym.

5. Generalna zasada jest taka, że należy obliczyć odwrotną transformatę Fouriera filtru idealnego.
Aby zagwarantować jednak dobrą lokalizację i symetryczność tworzonego filtru wykorzystuje się dodatkowe rotacje i przesunięcia.

        FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
        FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
        FilterFidft = cv2.idft(np.float32(FilterFRot3),
                        flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
        FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

6. Aby otrzymać poprawny filtr należy teraz pomnożyć dwuwymiarowe okno ze stworzonym filtrem idealnym w dziedzinie przestrzennej.

7. Stworzony filtr powinien zostać znowu przeniesiony do dziedziny częstotliwości.
Należy jednak rozszerzyć go tak, aby miał taki sam rozmiar jak filtrowany obraz.
Operację tę wykonaj tak samo jak w poprzednim ćwiczeniu (`cv2.copyMakeBorder`).
Oblicz transformatę Fouriera i wykonaj przesunięcie.
Następnie oblicz wartość bezwzględną (dzięki temu filtr nie będzie wpływał na fazę).
Wynikiem jest poprawiony filtr w dziedzinie częstotliwości.

8. Przeprowadź filtrację obrazu *lena.bmp* za pomocą zaprojektowanego filtra.
Wyświetl filtr, oraz wynik filtracji.
Do wyświetlenia filtru możesz użyć przestrzeń częstotliwości wygenerowaną w poprzednim zadaniu (*FSpaceRowsM* i *FSpaceColsM*).

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np
import math

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/'

fileNames = ["lena.bmp"]
for fileName in fileNames:
  if not os.path.exists(fileName):
      r = requests.get(url + fileName, allow_redirects=True)
      open(fileName, 'wb').write(r.content)


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Step 1: Define filter size
filter_size = 21

# Step 2: Create Hamming window
hamming = np.hamming(filter_size)

# Step 3: Create 2D window
hamming2d = np.outer(hamming, hamming)

# Step 4: Create ideal filter
# Assuming the ideal filter is a low-pass filter with a cutoff frequency
cutoff_frequency = 10
ideal_filter = cv2.getGaussianKernel(filter_size, cutoff_frequency)
ideal_filter = np.outer(ideal_filter, ideal_filter.transpose())

# Step 5: Compute inverse Fourier transform of the ideal filter
FilterFRot = np.rot90(np.fft.fftshift(np.rot90(ideal_filter, 2)), 2)
FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
FilterFidft = cv2.idft(np.float32(FilterFRot3), flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

# Step 6: Multiply 2D window with ideal filter
filter2d = hamming2d * FilterFI

# Step 7: Transfer the filter to frequency domain
# Assuming the image size is 512x512
image_size = 512
filter2d = cv2.copyMakeBorder(filter2d, 0, image_size - filter_size, 0, image_size - filter_size, cv2.BORDER_CONSTANT)
filter2d_dft = np.fft.fftshift(np.abs(np.fft.fft2(filter2d)))

# Step 8: Filter the image and display the filter and the result
image = cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)
image_dft = np.fft.fftshift(np.fft.fft2(image))
filtered_image_dft = image_dft * filter2d_dft
filtered_image = np.abs(np.fft.ifft2(np.fft.ifftshift(filtered_image_dft)))

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(filter2d_dft, cmap='gray')
plt.title('Filter')
plt.subplot(1, 3, 2)
plt.imshow(image, cmap='gray')
plt.title('Original Image')
plt.subplot(1, 3, 3)
plt.imshow(filtered_image, cmap='gray')
plt.title('Filtered Image')
plt.show()